In [29]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper
# BravyiKitaevMapper
from qiskit.circuit import QuantumCircuit, ParameterVector
# Parameter
# from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import spsa
from qiskit.circuit.library import EfficientSU2
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.primitives import BackendEstimatorV2
import scipy.sparse as sp
from qiskit.circuit.library import SwapGate, ExcitationPreserving
# from scipy.linalg import expm
from qiskit.quantum_info import Operator
# Clifford, Pauli, PauliList, SparsePauliOp
from qiskit.circuit.library import EvolvedOperatorAnsatz, HamiltonianGate, XXPlusYYGate
# PauliEvolutionGate, UnitaryGate
from qiskit.quantum_info import SparsePauliOp
# Pauli, PauliList,

In [30]:
n = 4
N = 2*n
t = 1
mu = 1.5
U = 3

In [31]:
def qOp(i,j):
    return FermionicOp(
    {
        "+_{i} -_{j}".format(i=i%N,j=j%N): 1.0,
    },
    num_spin_orbitals=N,
)

In [32]:
t_list = []
U_list = []
for i in range(n):
    t_list.append((2*i,(2*i+2)%N))
    t_list.append((2*i+1,(2*i+3)%N))
site_list = [2*i for i in range(n)]

t_term = 0
U_term = 0
mu_term = 0

for edge in t_list:
    t_term += qOp(edge[0],edge[1])
    t_term += qOp(edge[1],edge[0])

for u in site_list:
    mu_term += qOp(u,u) + qOp(u+1,u+1)
    U_term += qOp(u,u)@qOp(u+1,u+1)

H = U * U_term - mu * mu_term - t * t_term

U_list = [(i,i+1) for i in site_list]

In [33]:
def makePauliStr(N,idx,gs):
    pauli_str = ""
    for i in range(N):
        if i in idx:
            pauli_str+=gs[idx.index(i)]
        else:
            pauli_str+="I"
    return pauli_str

def ZZ(N,i,j):
    zz_str = makePauliStr(N,(i,j),["Z","Z"])
    return SparsePauliOp([zz_str],coeffs = [.5])

def dZZ(N,i,j,k,l):
    zz_str = makePauliStr(N,(i,j,k,l),["Z","Z","Z","Z"])
    return SparsePauliOp([zz_str],coeffs = [.5])

def XXplusYY(N,i,j):
    xx_str = makePauliStr(N,(i,j),["X","X"])
    yy_str = makePauliStr(N,(i,j),["Y","Y"])
    return SparsePauliOp([xx_str,yy_str],coeffs = [.25,.25])

def dXXplusYY(N,i,j,k,l):
    ij = XXplusYY(N,i,j)
    kl = XXplusYY(N,k,l)
    return ij@kl

In [34]:
def poolXXYY(N,ts):
    pool = []
    for i,edge in enumerate(ts):
        p = XXplusYY(N,edge[0],edge[1])
        # print(isinstance(p,Operator))
        pool.append(p)
    return pool

def pooldXXYY(N,ts):
    pool = []
    for i,edge in enumerate(ts):
        for j in range(i):
            p = dXXplusYY(N,edge[0],edge[1],ts[j][0],ts[j][1])
            # print(isinstance(p,Operator))
            pool.append(p)
    return pool

def poolZZ(N,Us):
    pool = []
    for edge in Us:
        p = ZZ(N,edge[0],edge[1])
        pool.append(p)
    return pool

def pooldZZ(N,Us):
    pool = []
    for i,edge in enumerate(Us):
        for j in range(i):
            p = dZZ(N,edge[0],edge[1],Us[j][0],Us[j][1])
            # print(isinstance(p,Operator))
            pool.append(p)
    return pool

def totalPool(N,Us,ts):
    ZZ = poolZZ(N,Us)
    dZZ = pooldZZ(N,Us)
    XXYY = poolXXYY(N,ts)
    dXXYY = pooldXXYY(N,ts)
    return XXYY + ZZ + dXXYY + dZZ

pool = totalPool(N,t_list,U_list)

In [35]:
# eZZ gate
theta = 0.7809
Smat = SwapGate().to_matrix()
Sop = Operator(SwapGate())
# print(Sop)
# expS = expm(1j*Smat)
# def expSt(t):
#   return np.power(expS,t)

def eSwap(circ,i,j,param):
    eSt = HamiltonianGate(Sop,param,label = "eSWAP")
    circ.append(eSt,[i,j])


def eZZ(circ,i,j,t):
    circ.cx(i,j)
    circ.rz(t,j)
    circ.cx(i,j)

def fij(circ,i,j,t):
    circ.append(XXPlusYYGate(t),[i,j])
    circ.cp(t,i,j)


def efSwap(circ,i,j,t):
    for k in range(j-1,i,-1):
        circ.cz(j,k)
    fij(circ,i,j,t)
    for k in range(i+1,j,1):
        circ.cz(j,k)

def hopping(circ,r):
    n = circ.num_qubits
    t_s = ParameterVector("t_h{r}".format(r=r),length = n)
    for i in range(n):
        efSwap(circ,i,(i+2)%n,t_s[i])

def interaction(circ,r):
    n = circ.num_qubits
    t_s = ParameterVector("t_i{r}".format(r=r),length = n//2)
    for i in range(0,n,2):
        eZZ(circ,i,i+1,t_s[i//2])


li = [i for i in range(0,n,4)]+[i+1 for i in range(0,n,4)]
def start(circ):
    n = circ.num_qubits
    for i in li:
        circ.x(i)
        circ.h(i+2)
        circ.cx(i,i+2)
    circ.barrier()

def start2(circ):
    n = circ.num_qubits
    li = [i for i in range(0,n//2,2)]+[n-1-i for i in range(0,n//2,2)]
    circ.x(li)
    circ.barrier()

def makePauliStr(N,idx,gs):
    pauli_str = ""
    for i in range(N):
        if i in idx:
            pauli_str+=gs[idx.index(i)]
        else:
            pauli_str+="I"
    return pauli_str

def ZZ(N,i,j):
    zz_str = makePauliStr(N,(i,j),["Z","Z"])
    return SparsePauliOp([zz_str],coeffs = [.5])

def dZZ(N,i,j,k,l):
    zz_str = makePauliStr(N,(i,j,k,l),["Z","Z","Z","Z"])
    return SparsePauliOp([zz_str],coeffs = [.5])

def XXplusYY(N,i,j):
    xx_str = makePauliStr(N,(i,j),["X","X"])
    yy_str = makePauliStr(N,(i,j),["Y","Y"])
    return SparsePauliOp([xx_str,yy_str],coeffs = [.25,.25])

def dXXplusYY(N,i,j,k,l):
    ij = XXplusYY(N,i,j)
    kl = XXplusYY(N,k,l)
    return ij@kl

def hopint(circ,reps = 3):
    for r in range(reps):
        hopping(circ,r)
        interaction(circ,r)
        # circ.append(EfficientSU2(num_qubits = circ.num_qubits, parameter_prefix="t_e{r}".format(r=r)),[i for i in range(circ.num_qubits)])
    
def hopint1(circ,reps = 3):
    for r in range(reps):
        hopping(circ,r)
        interaction(circ,r)

In [36]:
ans_hop = QuantumCircuit(N)
start2(ans_hop)
hopint(ans_hop, reps = 3)
ans_hop.draw(style = "mpl")

┌───┐ ░    ┌─────────────────────┐                   »
q_0: ┤ X ├─░────┤0                    ├─■─────────────────»
     └───┘ ░    │                     │ │                 »
q_1: ──────░──■─┤  (XX+YY)(t_h0[0],0) ├─┼────────────■────»
     ┌───┐ ░  │ │                     │ │P(t_h0[0])  │    »
q_2: ┤ X ├─░──■─┤1                    ├─■────────────■──■─»
     └───┘ ░    └─────────────────────┘                 │ »
q_3: ──────░────────────────────────────────────────────■─»
           ░                                              »
q_4: ──────░──────────────────────────────────────────────»
     ┌───┐ ░                                              »
q_5: ┤ X ├─░──────────────────────────────────────────────»
     └───┘ ░                                              »
q_6: ──────░──────────────────────────────────────────────»
     ┌───┐ ░                                              »
q_7: ┤ X ├─░──────────────────────────────────────────────»
     └───┘ ░                                              »
«                                                                      »
«q_0: ─────────────────────────────────────────────────────────────────»
«     ┌─────────────────────┐                                          »
«q_1: ┤0                    ├─■────────────────────────────────────────»
«     │                     │ │                 ┌─────────────────────┐»
«q_2: ┤  (XX+YY)(t_h0[1],0) ├─┼────────────■────┤0                    ├»
«     │                     │ │P(t_h0[1])  │    │                     │»
«q_3: ┤1                    ├─■────────────■──■─┤  (XX+YY)(t_h0[2],0) ├»
«     └─────────────────────┘                 │ │                     │»
«q_4: ────────────────────────────────────────■─┤1                    ├»
«                                               └─────────────────────┘»
«q_5: ─────────────────────────────────────────────────────────────────»
«                                                                      »
«q_6: ─────────────────────────────────────────────────────────────────»
«                                                                      »
«q_7: ─────────────────────────────────────────────────────────────────»
«                                                                      »
«                                                                 »
«q_0: ────────────────────────────────────────────────────────────»
«                                                                 »
«q_1: ────────────────────────────────────────────────────────────»
«                                                                 »
«q_2: ─■───────────────────────────────────────────────────────■──»
«      │                 ┌─────────────────────┐             ┌─┴─┐»
«q_3: ─┼────────────■────┤0                    ├─■───────────┤ X ├»
«      │P(t_h0[2])  │    │                     │ │           └───┘»
«q_4: ─■────────────■──■─┤  (XX+YY)(t_h0[3],0) ├─┼─────────────■──»
«                      │ │                     │ │P(t_h0[3])   │  »
«q_5: ─────────────────■─┤1                    ├─■─────────────■──»
«                        └─────────────────────┘                  »
«q_6: ────────────────────────────────────────────────────────────»
«                                                                 »
«q_7: ────────────────────────────────────────────────────────────»
«                                                                 »
«                                                              »
«q_0: ─────────────────────────────────────────────────────────»
«                                                              »
«q_1: ─────────────────────────────────────────────────────────»
«                                                              »
«q_2: ──────────────────────────■──────────────────────────────»
«     ┌─────────────┐         ┌─┴─┐                            »
«q_3: ┤ Rz(t_i0[1]) ├─────────┤ X ├────────────────────────────»
«     └─────────────┘┌────────┴───┴────────┐                   »
«q_